In [1]:
# First: load the model with vllm:

# vllm serve unsloth/gemma-3-1b-it --port 8008 --max-model-len 2500 --served-model-name unsloth/gemma-3-1b-it --gpu-memory-utilization 0.8 --dtype bfloat16 --api-key xxx

In [10]:
import subprocess
from pathlib import Path
from collections import Counter

In [13]:
# Example 1: Direct prompting mode. Returns a single unslopped output.

if True:
    cmd = [
        'python3', 'main.py',
        #'--api-base-url', 'http://localhost:8000/v1',
        '--api-key', 'xxx',
        '--model-name', 'Qwen/Qwen3-4B',
        '--chat-template-model-id', 'Qwen/Qwen3-4B',
        '--logging-level', 'INFO',
        '--slop-phrases-file', 'banlists/slop_phrases.json',
        '--top-n-slop-phrases', '500',
        '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
        '--max-new-tokens', '500',
        '--prompt', 'Write a story about a princess. '
    ]
    print('Running:', ' '.join(cmd))
    subprocess.run(cmd, check=True)

Running: python3 main.py --api-key xxx --model-name Qwen/Qwen3-4B --chat-template-model-id Qwen/Qwen3-4B --logging-level INFO --slop-phrases-file banlists/slop_phrases.json --top-n-slop-phrases 500 --regex-blocklist-file banlists/regex_not_x_but_y.json --max-new-tokens 500 --prompt Write a story about a princess. 
INFO mode: Progress bar and ban events will be printed. Most logs suppressed. Effective script level: INFO
--- Generation start ---

--- Generated Output (stream) ---
In the heart of the Enchanted Vale, where the skies were a canvas of ever-changing hues and the rivers sang lullabies to the forest, there lived a princess named Liora. She was not the kind of princess that the kingdom expected—no crown of gold, no royal guards, no grand palace. Liora was a simple girl who spent her days tending to the gardens, listening to the wind, and learning the ancient songs of the land.

Her father, King Alden, was a wise ruler who believed that true power came from understanding the worl

In [14]:
# Example 2: Generate an unslopped dataset using prompts from a huggingface sharegpt dataset

# Make sure you've launched the vllm openai-compatible server first. You can use something like:

# vllm serve unsloth/gemma-3-4b-it --port 8000 --max-model-len 2500 --served-model-name unsloth/gemma-3-4b-it --gpu-memory-utilization 0.95 --dtype bfloat16 --api-key xxx


THREADS  = 40
MAX_PROMPTS = 40

OUT_JSONL     = 'creative_writing_generations.jsonl'
HUMAN_PROFILE = 'data/human_writing_profile.json'


cmd = [
    'python3', 'main.py',
    #'--api-base-url', 'http://localhost:8000/v1',
    '--api-key', 'xxx',
    #'--model-name', 'unsloth/gemma-3-4b-it',
    '--model-name', 'Qwen/Qwen3-4B',
    '--output-jsonl', OUT_JSONL,
    '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT',
    '--hf-dataset-split', 'train',
    '--threads', str(THREADS),
    '--max-prompts', str(MAX_PROMPTS),
    '--logging-level', 'INFO',
    '--slop-phrases-file', 'banlists/slop_phrases.json',
    '--top-n-slop-phrases', '500',
    '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
    '--max-new-tokens', '2000',
    '--request-mode', 'chunk',  # chunk / stream (stream mode might be buggy)
    '--chunk-size', '20', # how many tokens we request from the api before checking it for new violations
    '--chat-template-model-id', "Qwen/Qwen3-4B",
    '--tdpo-pairs-jsonl', 'tdpo-pairs.jsonl'
]
print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True)


Running: python3 main.py --api-key xxx --model-name Qwen/Qwen3-4B --output-jsonl creative_writing_generations.jsonl --input-hf-dataset Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT --hf-dataset-split train --threads 40 --max-prompts 40 --logging-level INFO --slop-phrases-file banlists/slop_phrases.json --top-n-slop-phrases 500 --regex-blocklist-file banlists/regex_not_x_but_y.json --max-new-tokens 2000 --request-mode chunk --chunk-size 20 --chat-template-model-id Qwen/Qwen3-4B --tdpo-pairs-jsonl tdpo-pairs.jsonl
INFO mode: Progress bar and ban events will be printed. Most logs suppressed. Effective script level: INFO
Extracting HF prompts:   0%|          | 39/177477 [00:00<00:05, 34693.08prompt/s]
Preparing to process 40 new prompts in this run.
BANNED (prompt_idx=13, type=slop_phrase, phrase='laced')                  
BANNED (prompt_idx=18, type=ngram, ngram='felt weight')                   
BANNED (prompt_idx=0, type=slop_phrase, phrase='casting long shadows')    
BANNED (prompt_idx=

2025-05-20 23:25:39,110 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates found.


BANNED (prompt_idx=23, type=slop_phrase, phrase='unseen')                 
BANNED (prompt_idx=12, type=slop_phrase, phrase='shifted')                
BANNED (prompt_idx=36, type=slop_phrase, phrase='void')                   
BANNED (prompt_idx=25, type=slop_phrase, phrase='rhythm')                 
BANNED (prompt_idx=20, type=ngram, ngram='left behind')                   
BANNED (prompt_idx=17, type=ngram, ngram='eyes fixed')                    
BANNED (prompt_idx=38, type=ngram, ngram='air around')                    
BANNED (prompt_idx=39, type=slop_phrase, phrase='villain')                
BANNED (prompt_idx=0, type=slop_phrase, phrase='air was thick with the scent')
Batch Generating:   0%|          | 0/40 [00:04<?, ?prompt/s, 1059.0 tok/s]

2025-05-20 23:25:39,731 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates found.


BANNED (prompt_idx=27, type=slop_phrase, phrase='fabric')                 
BANNED (prompt_idx=33, type=slop_phrase, phrase='froze')                  
BANNED (prompt_idx=3, type=slop_phrase, phrase='echo')                    
BANNED (prompt_idx=15, type=slop_phrase, phrase='echo')                   
BANNED (prompt_idx=19, type=ngram, ngram='heart pounded')                 
BANNED (prompt_idx=31, type=slop_phrase, phrase='countless')              
BANNED (prompt_idx=12, type=slop_phrase, phrase='shimmered')              
BANNED (prompt_idx=35, type=slop_phrase, phrase='relentless')             
BANNED (prompt_idx=25, type=slop_phrase, phrase='rhythm')                 
BANNED (prompt_idx=2, type=slop_phrase, phrase='curse')                   
BANNED (prompt_idx=20, type=ngram, ngram='one day')                       
Batch Generating:   0%|          | 0/40 [00:05<?, ?prompt/s, 1009.2 tok/s]

...

2025-05-20 23:26:41,472 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates found.


BANNED (prompt_idx=15, type=slop_phrase, phrase='smiled')                         
BANNED (prompt_idx=33, type=ngram, ngram='first time long')                       
BANNED (prompt_idx=22, type=regex, regex='\bnot\s+(?:just|only|merely)?\s*(?:(?!but\b|[.?!]).){1,50}?[,;:—–-]?\s*but\s+(?:also\s+)?', match='not just fear, but ')
Batch Generating:  82%|████████▎ | 33/40 [01:06<00:07,  1.07s/prompt, 240.7 tok/s]

2025-05-20 23:26:42,077 [ERROR] [core.sampler        ]: Back-track: no valid next-token candidates found.


BANNED (prompt_idx=15, type=slop_phrase, phrase='nodded')                         
BANNED (prompt_idx=22, type=slop_phrase, phrase='something else entirely')        
BANNED (prompt_idx=15, type=slop_phrase, phrase='nodded')                         
BANNED (prompt_idx=33, type=slop_phrase, phrase='laced')                          
BANNED (prompt_idx=38, type=slop_phrase, phrase='shifted')                        
BANNED (prompt_idx=15, type=slop_phrase, phrase='nodded')                         
BANNED (prompt_idx=22, type=slop_phrase, phrase='knew one thing')                 
BANNED (prompt_idx=33, type=regex, regex='\bnot\s+(?:just|only|merely)?\s*(?:(?!but\b|[.?!]).){1,50}?[,;:—–-]?\s*but\s+(?:also\s+)?', match='Not a perfect one, not a gourmet one, but ')
BANNED (prompt_idx=15, type=regex, regex='\bnot\s+(?:just|only|merely)?\s*(?:(?!but\b|[.?!]).){1,50}?[,;:—–-]?\s*but\s+(?:also\s+)?', match='Not pride, not honor, but ')
Batch Generating: 100%|██████████| 40/40 [01:35<00:00,  2.38s/p

CompletedProcess(args=['python3', 'main.py', '--api-key', 'xxx', '--model-name', 'Qwen/Qwen3-4B', '--output-jsonl', 'creative_writing_generations.jsonl', '--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT', '--hf-dataset-split', 'train', '--threads', '40', '--max-prompts', '40', '--logging-level', 'INFO', '--slop-phrases-file', 'banlists/slop_phrases.json', '--top-n-slop-phrases', '500', '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json', '--max-new-tokens', '2000', '--request-mode', 'chunk', '--chunk-size', '20', '--chat-template-model-id', 'Qwen/Qwen3-4B', '--tdpo-pairs-jsonl', 'tdpo-pairs.jsonl'], returncode=0)